In [105]:
import pandas as pd
import numpy as np
import heapq
import pdb
import math, sys
from scipy.stats import chi2_contingency, chi2
from collections import defaultdict
from itertools import combinations, chain

In [110]:
## Set up test case for particular node
df = pd.read_csv("tests/data-strikes2/data.txt", delim_whitespace=True)
GOTO_NODE = True
if GOTO_NODE:
    _df = df.loc[df['num1'] <= 1.5]
    _df = _df.loc[_df['num2'] <= 0.4375]
    _df = _df.loc[_df['num1'] <= 0.5]
######################################################################################

col = 'cat1'
self_tgt = 'tgt'
node_y_mean = _df[self_tgt].mean()
_df = _df[[col,self_tgt]]
x_uniq = _df[col].drop_duplicates().sort_values().values

node_binom = _df[self_tgt] <= node_y_mean                      # class 1 has negative residual
_df.loc[:,self_tgt] = node_binom

def pi(j):
    if j == 1: return _df[self_tgt].sum() / len(_df)
    # if j == 1: return (df[self_tgt] < df[self_tgt].mean()).mean()
    if j == 0: return 1.0 - pi(1)
    raise "fail"
    
def N_j_s(j,s):
    """ num cases of class j in subset """
    assert j == 0 or j == 1
    return _df[(_df[self_tgt] == j) & (_df[col]== s)].shape[0]
    
def p(j, subset_l) -> float:
    assert j == 0 or j == 1
    return pi(j)*N_j_l(j, subset_l) / ( pi(1)*N_j_l(1, subset_l) + pi(0)*N_j_l(0, subset_l) )
    
sorted_ps = sorted([(s, p(1, (s,))) for s in x_uniq],key=lambda x:x[1])

sorted_ps = np.asarray(sorted_ps)   

st = 0
en = 0
subsets = [sorted_ps[st:en,0]for en in range(1,len(x_uniq))]

sorted_ps

array([['A', '0.5165635238441937'],
       ['J', '0.5231497127407908'],
       ['H', '0.5643044619422573'],
       ['G', '0.6812289966394622'],
       ['D', '0.7905551550108146'],
       ['E', '0.8103940605368362'],
       ['F', '0.8103940605368362'],
       ['I', '0.8131472309770374'],
       ['B', '0.8161974058842139'],
       ['C', '0.8614357262103507']], dtype='<U32')

In [107]:
for s in subsets:
    print(s)

['A']
['A' 'J']
['A' 'J' 'H']
['A' 'J' 'H' 'G']
['A' 'J' 'H' 'G' 'D']
['A' 'J' 'H' 'G' 'D' 'E']
['A' 'J' 'H' 'G' 'D' 'E' 'F']
['A' 'J' 'H' 'G' 'D' 'E' 'F' 'I']
['A' 'J' 'H' 'G' 'D' 'E' 'F' 'I' 'B']


In [108]:


for s in subsets:
   left_idx = _df[_df[col].isin(subset)].index.values
   right_idx = _df.drop(left_idx).index.values

   # gini impurity of left and right nodes based on residual sign
   mean_left = (_df.loc[left_idx, self.tgt] <= node.y_mean).mean() 
   mean_right = (_df.loc[right_idx, self.tgt] <= node.y_mean).mean() 
   mean_node  = (_df[self.tgt] <= node.y_mean).mean()

   Nall = node.idx.shape[0]
   p = 1, len(left_idx)/Nall, len(right_idx)/Nall
    
   gini_node = 2 * mean_node * (1 - mean_node)
   gini_left = 2 * mean_left * (1 - mean_left)
   gini_right = 2 * mean_right * (1 - mean_right)
   gain = p[0]*gini_node - p[1]*gini_left - p[2]*gini_right

   results['set'].append(subset)
   results['gain'].append(gain)


NameError: name 'subset' is not defined